## Diskussion einiger beispielhafter Ergebnisse

Im folgenden wollen wir uns die Ergebnisse für zwei Netzwerke anschauen, die mit den Notebooks in diesem Ordner trainiert und evaluiert worden sind, und die sich in einem einzigen Parameter unterscheiden, nämlich der Lernrate.

### Cave

- Die Anzahl an verfügbaren Röntgen-Thoraxaufnahmen hat sich seit der Erstellung dieser Notebooks deutlich erhöht. Die Diskussion bezieht sich noch auf den Stand im Sommer 2020, als noch *sehr wenige* Röntgenthoraxaufnahmen von COVID-19 Pneumonien frei verfügbar waren. Wundern sie sich also nicht, falls die Zusammensetzung ihres Datensatzes nicht den in der Diskussion genannten Zahlenwerten entspricht.

- Des weiteren wurde die Standardlernrate in dem Trainingsnotebook von 0.001 auf 0.0001 reduziert, d.h. das unten diskutierte Netzwerk 2 entspricht nun den vorgeschlagenen Standardparametern (warum sollte im Verlauf der Diskussion klar werden).

### Datensatz

Erinnern wir uns noch einmal an die Zusammensetzung unseres Datensatzes.

```
Anzahl Bilder "normal": 
8851

Anzahl Bilder "pneumonia":
6069

Number of COVID Images:
617
```

Betrachten wir ausserdem ein paar zufällig gezogene Beispiele für die drei Klassen:

#### Normalbefunde
<img src="./images/training_examples/normal.png"/>

#### Bakterielle Pneumonien
<img src="./images/training_examples/pneumonia.png"/>

#### COVID-19
<img src="./images/training_examples/covid.png"/>

Sie sehen, dass in diesem Datensatz nicht nur die radiologischen Eigenschaften des Lungengewebes eine Rolle spielen, die man sehr verkürzt so zusammen fassen könnte:

- Normalbefunde: Lunge überall frei belüftet
- bakterielle Pneumonien: Fokale oder lobuläre Verschattungen, oft in basalen Lungenabschnitten.
- COVID-19: Milchglastrübungen, eher peripher und bilateral (s. Pfeile im linken, mit "B" markierten Beispielbild).


Allerdings erkennen sie auch andere Eigenschaften der Bilder, die mit der entsprechenden Klasse korrelieren:

- Die Normalbefunde, die sie sehen, sind alle __im Stehen__ aufgenommen. 
- Sowohl Patienten mit bakterielle Pneumonien als auch mit schweren COVID-Verläufen haben deutlich öfter __Liegendaufnahmen__ erhalten, da diese Populationen einfach oft __zu stark erkrankt__ waren, um für eine Röntgenaufnahme aufzustehen. 
- Dementsprechend sieht man auf den Bildern der Klassen "covid" und "pneumonia" auch öfter Artefakte, die auf eine schwerere Erkrankung hinweisen, wie z.B. __EKG Elektroden__.
- Die Bilder der schweren COVID-Verläufe wurden zudem zu einem großen Teil aus Publikationen gesammelt und enthalten entsprechende Features, wie __Abbildungsnummerierungen und Pfeile__, die man in Abbildungen in wissenschaftlichen Artikeln findet.

### Netzwerk 1

Dieses Netzwerk wurde mit dem Notebook 01_Train_CNN_on_XRays.ipynb in diesem Ordner trainiert, wobei die voreingestellten Standardoptionen beibehalten wurden. Dazu zählen insbesondere

```
Architektur: 'squeezenet'
Lernrate: 0.001
Optimizer: Adam
Split Trainings-/Validierungs-/Testset: 0.8/0.1/0.1
```

Die voreingestellte Architektur, SqueezeNet, unterscheidet sich von den anderen vier möglichen Architekturen deutlich in der Anzahl der freien Parameter (i.e. der lernbaren Netzwerkgewichte und Biases). Bei SqueezeNet sind das ca. 700.000 Parameter. Bei allen anderen Architekturen mehrere zehn Millionen, bei VGG z.B. ca. 130 Millionen. SqueezeNet wurde eigentlich entwickelt, um Netzwerke auch direkt auf mobilen Endgeräten benutzen zu können, so dass ihr Mobiltelefon z.B. Gegenstände auf Bildern erkennen kann, ohne diese zunächst "in die Cloud" schicken und dort verarbeiten zu müssen.

Für uns hatte diese Architektur den Vorteil, dass sie aufgrund der relativ Kleinen Anzahl der Parameter relativ schnell trainiert werden kann und wir in den wenigen Stunden, die wir heute zur Verfügung hatten, schon erste Ergebnisse sehen konnten.

#### Lernkurven

Betrachten wir zunächst die Lernkurven

##### Kostenfunktion
<img src="./images/standard/loss.png">

##### Accuracy
<img src="./images/standard/accuracy.png">

Wir können Ablesen, dass unser Netzwerk für 9 Epochen trainiert wurde, d.h. jeden Datenpunkt im Trainingsset 9 mal gesehen hat. Wir sehen, dass die Kostenfunktion auf dem Validierungsset nach wie vor fällt, d.h. dass das Netzwerk noch nicht austrainiert ist und auch nicht überfittet wurde. Dies ist bei den wenigen Stunden, die wir heute trainiert haben, nicht verwunderlich. Würde man länger trainieren, üblich sind ca. 100 Epochen, würde die Kostenfunktion auf dem Validierungsset irgendwann ein Plateau erreichen bzw. sogar wieder ansteigen, sobald das Netzwerk beginnen würde, sich nicht nur die Struktur sondern auch das zufällige Rauschen im Trainingsdatensatz zu merken und somit zu "überfitten". 

Wir können also zwei Dinge ablesen:

- Das Validierungsloss sinkt kontinuierlich, springt nicht stark hin- und her und explodiert auch nicht. D.h. prinzipiell sind die Parameter für den Lernalgorithmus nicht ganz schlecht gewählt.
- Das Validierungsloss ist gerade noch am sinken. D.h. das Training ist noch nicht abgeschlossen und das Netzwerk würde noch weiter sinnvolle Strukturen aus den Daten lernen, wenn man das training noch für einige Epochen, d.h. komplette Durchgänge durch den Trainingsdatensatz, fortsetzen würde. In der Regel trainiert man für ca. 100 Epochen und benutzt ein Early-Stopping-Verfahren, dass das Training beendet, sobald das Validierungsloss über mehrere Epochen hinweg konstant ansteigt.

Ganz ähnlich sehen wir auch, dass die Accuracy sowohl auf dem Trainings-, als auch auf dem Validierungsset noch ansteigt.

__Aber Vorsicht__: Erinnern sie sich an die Zusammensetzung unseres Datensatzes. Nur $617$ der $(617 + 6069 + 8851)= 15537$ Bilder, also ca. 4% der Bilder gehören zur Klasse COVID. D.h. sogar wenn ein trainiertes Netzwerk alle Bilder dieser Klasse komplett falsch zuordnen würde, könnte es theoretisch noch eine Accuracy von 96% erreichen, wenn es alle Normalbefunde und bakteriellen Pneumonien richtig klassifiziert.

#### Metriken auf dem Testset

Betrachten wir nun die Ergebniss-Metriken auf dem Testset:
    
```
------------------ Test Metrics -----------------------

Accuracy 0.894

Recall on normal test data 0.966

Recall on pneumonia test data 0.868

Recall on COVID test data 0.129

Test Loss 0.34328509752567

------------------------------------------------------------
```

Hier muss nochmals __betont__ werden, dass für sogenannte 'unbalancierte' Datensätze, in denen die einzelnen Klassen unterschiedlich oft vorkommen, die Accuracy kein sehr sinnvolles Performancemaß ist.

Stattdessen sollten wir eher auf den "Recall", d.h. die Sensitivität bezüglich der einzelnen Klassen, schauen. Hier sehen wir, dass von den Testbildern, die tatsächlich einen Normalbefund zeigen, ca. 97% auch von Netzwerk als 'normal' klassifiziert werden. Von den Bildern, die tatsächlich eine bakterielle Pneumonie zeigen, klassifiziert das Netzwerk knapp 87% als 'pneumonia'. Von den Bildern, die tatsächlich eine COVID-Erkrankung darstellen, erkennt das Netzwerk jedoch nur __knapp 13%__.

Diese Größen finden sich auch in der Confusion-Matrix wieder.

#### Confusion-Matrix
<img src="./images/standard/confusion_matrix.png">

Aus dieser Matrix kann man die entsprechenden Recalls ("Sensitivitätswerte") bezüglich der einzelnen Klassen berechnen, in dem man die Anzahl der korrekt klassifizierten Bilder dieser Klasse durch die gesamtzahl der Bilder dieser Klasse (entsprechende Zeilensumme) teilt.

Andersherum können wir auch die "Precision" (analog zum "Positiven Prädiktiven Wert") für die einzelnen Klassen berechnen, also die Wahrscheinlichkeit, dass ein Datenpunkt tatsächlich die Klasse hat, die vom Netzwerk vorhergesagt wird. Hierzu teilt man die Anzahl der korrekt klassifizierten Bilder dieser Klasse durch die gesamtzahl der Bilder, die vom Netzwerk dieser Klasse zugeordnet werden (entsprechende Spaltensumme). Im Fall der COVID Klasse ist dieser Wert hier tatsächlich "1", da alle 8 Bilder, die vom Netzwerk der Klasse COVID zugeordnet worden sind, auch tatsächlich zu dieser Klasse gehören.

#### Feature Attribution mit GradCAM

Um einen kleinen Einblick in die "Black-Box" zu bekommen, die ein trainiertes tiefes neuronales Netzwerk ist, können wir uns anschauen, welche Pixel eines konkreten Bildes man verändern müsste, um die entsprechende Entscheidung des Netzwerkes maximal zu beeinflussen. Ein Algorithmus, der uns dies erlaubt, ist GradCAM. Im folgenden sehen sie die Ergebnisse für einige Beispielbilder, die ihren entsprechenden richtigen Klassen zugeordnet wurden. Eine "Heatmap" zeigt an, wie empfindlich der Output des neuronalen Netzwerkes auf Änderungen der Grauwerte der entsprechenden Pixel __dieses konkreten Bildes__ reagieren würde. Rot bedeutet, dass Veränderungen dieser Pixel die Vorhersage des Netzwerkes __für dieses konkrete Bild__ stark beinflussen würden, blau bedeutet, dass die Vorhersage des Netzwerkes __für dieses konkrete Bild__ sehr unempfindlich gegenüber Änderungen der entsprechenden Pixel ist.

#### 'normal' klassifiziert als 'normal'
<img src="./images/standard/normal_normal.png"/>

Wie sie sehen, achtet das trainierte Netzwerk hier hauptsächlich auf den belüfteten Thorax. Man kann sich überlegen, dass es wohl sinnvoll wäre, die Vorhersage "Normalbefund" zu ändern, sobald irgendwo in diesen Bereichen z.B. Verschattungen auftreten würden.

#### 'pneumonia' klassifiziert als 'pneumonia'
<img src="./images/standard/pneumonia_pneumonia.png"/>

Wenn es darum geht, Bilder der Klasse der bakteriellen Pneumonien zu erkennen, konzentriert sich das Netz zum einen auf  basale, verschattete Lungenabschnitte, jedoch auch auf Artefakte, die eher mit einem schwereren Krankheitsverlauf korrelieren, wie z.B. EKG-Elektroden. Hin und wieder beachtet es jedoch auch apikale Lungenabschnitte, sowie die Ecken und den oberen Rand der Aufnahme, wo man hin und wieder Beschriftungen findet. Beachten sie zudem, dass das Netzwerk auch zum Teil auf die Position der __Arme__ des Patienten achtet, ein Hinweis darauf, ob es sich um eine __Stehend- oder Liegendaufnahme__ handelt.

#### 'covid' klassifiziert als 'covid'
<img src="./images/standard/covid_covid.png"/>

Um die COVID-Bilder zu erkennen, benutzt das Bild einzelne bis wenige, recht zufällig lokalisierte Pixel. Die starke Abhängigkeit von solchen sehr lokalen, und eher zufällig verteilten Bildeigenschaften spricht dafür, dass dieses Netzwerk sich eher zufälliges Rauschen in den COVID-Trainingsbildern gemerkt hat, als die tatsächliche Struktur in den Bildern dieser Klasse zu lernen (z.B. Milglastrübungen). Das würde auch erklären, warum die entsprechende Sensivitität für die Klasse "covid" so gering ist, im Vergleich zu den anderen beiden Klassen.

Betrachten sie die Confusion-Matrix und überlegen sie, warum für "covid" klassifiziert als "covid" nur 8 und nicht 10 Beispiele angezeigt werden.

<details><summary>Klicken sie <b>hier</b> für eine Erklärung</summary>
<p>

In der Confusion-Matrix können sie ablesen, dass von den 62 Bildern im Testdatensatz, die tatsächlich COVID-Befunde zeigen, nur 8 vom Netzwerk auch als COVID-Befunde klassifiziert wurden. D.h. sie sehen hier nicht wie bei den beiden anderen Klassen 10 zufällige Beispiele aus den 860 (normal als normal) bzw. 530 (pneumonia als pneumonia) korrekt klassifizierten Bildern, sondern *alle* korrekt klassifizierten Bilder für die Klasse 'covid'.

Wie könnte dies mit der schlechten Performance des Netzwerks im Bezug auf diese Klasse zusammenhängen

<details><summary>Klicken sie <b>hier</b> für eine Erklärung</summary>
<p>

Bei dem Datensatz, den wir ihnen bereit gestellt haben, handelt es sich um einen __sehr unbalancierten__ Datensatz. Von den 3 Klassen, die der Datensatz beinhaltet, machen 2 Klassen, nämlich normalbefunde und bakterielle Pneumonien, 96% der Bilder aus. D.h. das Netzwerk sieht mehr als __zehnmal__ mehr Beispiele von Normalbefunden und bakteriellen Pneumonien, als von COVID-Befunden. Da es sich bei COVID-19 allerdings um eine sehr neue Erkrankung handelt, und es im Vergleich zu Normalbefunden und bakteriellen Pneumonien sehr wenige Daten gibt, lässt sich so eine Situation jedoch hin und wieder nicht verhindern. Dann ist es umso wichtiger, gut auf ein entsprechendes Vorgehen beim Training und der Evaluierung von Machine-Learning-Methoden zu achten, um die Ergebnisse realistisch einschätzen zu können.

### Netzwerk 2

Hier haben wir lediglich die Lernrate von 0.001 auf 0.0001 reduziert.

```
Architektur: 'squeezenet'
Lernrate: 0.0001
Optimizer: Adam
Split Trainings-/Validierungs-/Testset: 0.8/0.1/0.1
```

#### Lernkurven

Betrachten wir zunächst wieder die Lernkurven

##### Kostenfunktion
<img src="./images/lr0001/loss.png">

##### Accuracy
<img src="./images/lr0001/accuracy.png">

Wir sehen hier vor allem im Anstieg der Accuracy den Effekt der kleineren Lernrate von 0.0001, die zu einem etwas langsameren Anstieg der Accuracy, verglichen mit einer Lernrate von 0.001, führt.

Ebenso sehen wir, dass dieses Netzwerk ein wenig länger trainiert wurde. Es hat jeden Datenpunkt nicht 9 mal, sondern 10 mal gesehen.

Analog zum ersten Netzwerk sieht man ebenfalls keine großen Sprünge und keine Explosion der Kurven, so dass auch hier das Training prinzipiell funktioniert.

Ebenso sieht man hier auch noch kein Plateau bzw. keinen systematischen Anstieg der Kostenfunktion auf dem Validierungsloss, so dass auch hier das Training noch nicht abgeschlossen ist und es deshalb sinnvoll wäre, das Netzwerktraining noch etliche Epochen (~100) weiterlaufen zu lassen.

#### Metriken auf dem Testset

Betrachten wir nun die Ergebniss-Metriken auf dem Testset:
    
```
------------------ Test Metrics -----------------------

Accuracy 0.905

Recall on normal test data 0.950

Recall on pneumonia test data 0.893

Recall on COVID test data 0.387

Test Loss 0.27642501120288404

------------------------------------------------------------
```

Wir sehen, dass die __kleinere Lernrate__ in diesem konkreten Fall anscheinend sehr __hilfreich__ war. Denn während der Recall für Normalbefunde und Befunde bei bakteriellen Pneumonien praktisch identisch ist, ist der Recall für Befunde bei COVID-Erkrankungen deutlich angestiegen. Nun werden immerhin __38,7%__ der Bilder, die COVID-Befunde zeigen, auch der Klasse 'covid' zugeordnet.

Wie oben kann man den Recall auch wieder aus den entsprechenden Einträgen der Confusion-Matrix berechnen.

#### Confusion-Matrix
<img src="./images/lr0001/confusion_matrix.png">

Können sie die Precision (d.h. den Positiven Prädiktiven Wert) für die Klasse 'covid' aus dieser Confusion-Matrix bestimmen? 

Wichtig: Erinnern sie sich, dass sich diese Kenngröße immer sowohl auf das Netzwerk als auch auf den ausgewerteten Datensatz bezieht, da diese Größe auch von den entsprechenden Vortestwahrscheinlichkeiten abhängt.

<details><summary>Klicken sie <b>hier</b> für die Lösung</summary>
<p>

Um die Precision bezüglich einer Klasse zu berechnen, teilen wir die Anzahl der korrekt klassifizierten Bilder dieser Klasse durch die Gesamtanzahl der Bilder, die vom Netzwerk dieser Klasse zugeordnet wurden (Spaltensumme). In unserem Fall erhalten wir damit:

$$\mathrm{ Precision('covid') } = \frac{24}{24 + 4 + 0} = 24/28 \approx 85,7\%$$

D.h. dass die Wahrscheinlichkeit, dass Bilder __aus dem Testdatensatz__, die das Netzwerk als 'covid' klassifiziert hat, wirklich COVID-Befunde zeigen 85,7% beträgt.

#### Feature Attribution mit GradCAM

Zuletzt untersuchen wir noch, ob dem gestiegenen Recall für die Klasse 'covid' auch wirklich das Erkennen sinnvoller Strukturen zu Grunde liegt. Dazu benutzwen wir wieder GradCAM, um zu sehen, welche Pixel man manipulieren müsste, um die Vorhersagen des trainierten Netzwerks für bestimmte Bilder maximal zu verändern.

#### 'normal' klassifiziert als 'normal'
<img src="./images/lr0001/normal_normal.png"/>

#### 'pneumonia' klassifiziert als 'pneumonia'
<img src="./images/lr0001/pneumonia_pneumonia.png"/>

#### 'covid' klassifiziert als 'covid'
<img src="./images/lr0001/covid_covid.png"/>

Hier sehen wir zum einen, dass das Netzwerk nun tatsächlich die Milchglastrübungen erkennt, die in der Literatur zur Bildgebung bei COVID auch beschrieben werden. Andererseits hat das Netzwerk jedoch auch gelernt, auf die Beschriftungen in der linken oberen Ecke der Abbildungen und auf die Pfeilmarkierungen zu schauen. Da dies jedoch tatsächlich Strukturen in unseren Trainings- und Testdaten darstellt, die mit der Klasse 'covid' korrelliert, ist dies nicht als Versagen des Netzwerkes bzw. Trainings zu bezeichnen. Ein flexibler Lernalgorithmus, so wie z.B. dieses tiefe neuronale Netz, lernt nun einmal *alles an Struktur* in den Trainingsdaten zu benutzen, was man ihm anbietet, um seine entsprechende Zielfunktion zu minimieren.

D.h. falls man das Netzwerk dazu bewegen möchte, zur Klassifizierung nur auf Veränderungen des Lungenparenchyms zu achten, muss man ihm auch einen entsprechenden Trainingsdatensatz präsentieren, der keine anderen Artefakte enthält, die mit dem Wert der Zielfunktion (hier: der Bildklasse) korrellieren. Dazu könnte man z.B. darauf achten, dass Bilder mit entsprechenden Artefakten aussortiert werden, oder dass - von Hand oder automatisch - Bereiche außerhalb der Lunge aus den Trainingsbildern weggeschnitten werden. Theoretisch wäre es auch möglich, die Daten so zu augmentieren, dass die Statistik der Artefakte über alle drei Klassen hinweg gleich ist, in dem man z.B. den Normalbefunden künstlich EKG-Kabel hinzufügt. So oder so sollte man immer im Hinterkopf behalten, dass lernende Systeme __jede Struktur__ in den Trainingsdaten erfassen und ausnutzen, die man Ihnen anbietet, sofern nicht entsprechende Invarianzen in der Netzwerk architektur kodiert sind. Ein Beispiel hierfür sind z.B. die frühen Layer von CNNs, deren Filter auf ähnliche lokale Muster unabhängig von ihrer genauen Position in einem Bild gleich reagieren, und somit eine Invarianz bezüglich der genauen Position eines Musters kodieren, eine sogenannte Translationsinvarianz. Durch eine Verkettung solcher Filter-Layer hintereinander, kann das Netzwerk weiterhin aus Muster lernen, die auf ähnlichen Größenskalen ähnlich sind. Somit kann man eine unabhängigkeit von der Gräße eines Objektes kodieren, eine sogennante Skaleninvarianz. Dennoch sollte unabhängig von der Netzwerkarchitektur immer darauf geachtet werden, dass man die Trainingsdaten entsprechend balanciert, vorverarbeitet oder augmentiert, so dass die Statistik der Trainingsdaten auch den Daten im Anwendungsfall entspricht und die Strukturen wiederspiegelt, die das Netzwerk lernen soll.